<a href="https://colab.research.google.com/github/mrp17010/cse4709-project/blob/main/temp/cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import network
import time
from machine import Pin
from umqtt.simple import MQTTClient

# Define the pins for the two cars
car1_forward = Pin(12, Pin.OUT)
car1_backward = Pin(13, Pin.OUT)
car2_forward = Pin(14, Pin.OUT)
car2_backward = Pin(15, Pin.OUT)

# Set the initial state of the pins to LOW
car1_forward.value(0)
car1_backward.value(0)
car2_forward.value(0)
car2_backward.value(0)

# Function to control car 1
def control_car1(message):
    if message == b'F':
        car1_forward.value(1)
        car1_backward.value(0)
    elif message == b'B':
        car1_forward.value(0)
        car1_backward.value(1)
    else:
        car1_forward.value(0)
        car1_backward.value(0)

# Function to control car 2
def control_car2(message):
    if message == b'F':
        car2_forward.value(1)
        car2_backward.value(0)
    elif message == b'B':
        car2_forward.value(0)
        car2_backward.value(1)
    else:
        car2_forward.value(0)
        car2_backward.value(0)

# Function to control both cars
def control_both_cars(message):
    if message == b'F':
        car1_forward.value(1)
        car1_backward.value(0)
        car2_forward.value(1)
        car2_backward.value(0)
    elif message == b'B':
        car1_forward.value(0)
        car1_backward.value(1)
        car2_forward.value(0)
        car2_backward.value(1)
    else:
        car1_forward.value(0)
        car1_backward.value(0)
        car2_forward.value(0)
        car2_backward.value(0)

# MQTT callback function
def sub_callback(topic, message):
    print(f'from {topic} got {message}')
    # Check the topic to determine which car to control
    if topic == b'm_patel/feeds/car1':
        control_car1(message)
    elif topic == b'm_patel/feeds/car2':
        control_car2(message)
    elif topic == b'm_patel/feeds/both_cars':
        control_both_cars(message)

# Connect to WiFi
wlan = network.WLAN(network.STA_IF)
wlan.active(True)
ssid = 'patel'
password = ''
wlan.connect(ssid, password)

while not wlan.isconnected():
    print('Waiting for connection...')
    time.sleep(1)
print("Connected to WiFi")

# MQTT parameters
mqtt_host = 'io.adafruit.com'
mqtt_username = 'm_patel'
mqtt_password = ''
mqtt_client_id = 'madhav_patel_test_mqtt'

# Connect to MQTT broker
client = MQTTClient(client_id=mqtt_client_id,
                    server=mqtt_host,
                    user=mqtt_username,
                    password=mqtt_password)

client.set_callback(sub_callback)
client.connect()

# Subscribe to topics for both cars and both cars at once
client.subscribe(b'm_patel/feeds/car1')
client.subscribe(b'm_patel/feeds/car2')
client.subscribe(b'm_patel/feeds/both_cars')
print("Connected and subscribed")

try:
    while True:
        client.wait_msg()

except Exception as e:
    print("Something went wrong")
    print(e)

finally:
    # Disconnect MQTT client
    client.disconnect()
    print("Disconnected")
